In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")
import opendatasets as od
import os
import cv2
from tqdm import *


In [2]:
od.download(dataset_id_or_url="https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces",data_dir="C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading 140k-real-and-fake-faces.zip to C:\Users\amith\Documents\Datasets\8 - Detecting DeepFakes with ResNet and CNN\140k-real-and-fake-faces


100%|██████████| 3.75G/3.75G [01:12<00:00, 55.1MB/s]


In [2]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)


In [3]:
train_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\train",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary",
)

val_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\valid",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary"
)

test_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\test",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary"
)


Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
model = tf.keras.models.Sequential(layers=[
    
    tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])


In [5]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(), loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])


In [6]:
model.fit(train_data,epochs=10,verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=2),validation_data=val_data,steps_per_epoch=100000/1000,validation_steps=20000/1000)


Epoch 1/10
100/100 [==============================] - 12s 109ms/step - loss: 0.6914 - accuracy: 0.4953 - val_loss: 0.6869 - val_accuracy: 0.4922
Epoch 2/10
100/100 [==============================] - 11s 107ms/step - loss: 0.6858 - accuracy: 0.5119 - val_loss: 0.6843 - val_accuracy: 0.5203
Epoch 3/10
100/100 [==============================] - 11s 104ms/step - loss: 0.6802 - accuracy: 0.5072 - val_loss: 0.6795 - val_accuracy: 0.5047
Epoch 4/10
100/100 [==============================] - 11s 108ms/step - loss: 0.6720 - accuracy: 0.5081 - val_loss: 0.6760 - val_accuracy: 0.4969
Epoch 5/10
100/100 [==============================] - 10s 103ms/step - loss: 0.6638 - accuracy: 0.5038 - val_loss: 0.6560 - val_accuracy: 0.4703
Epoch 6/10
100/100 [==============================] - 11s 111ms/step - loss: 0.6541 - accuracy: 0.4944 - val_loss: 0.6523 - val_accuracy: 0.4906
Epoch 7/10
100/100 [==============================] - 11s 113ms/step - loss: 0.6523 - accuracy: 0.5072 - val_loss: 0.6606 - val_ac

In [7]:
losses = model.evaluate(test_data,steps=100)


100/100 [==============================] - 7s 65ms/step - loss: 0.6243 - accuracy: 0.4916


In [9]:
losses


[0.624281108379364, 0.49156248569488525]

In [13]:
pred_data = model.predict(test_data)


625/625 [==============================] - 94s 150ms/step


In [16]:
pred_data


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [18]:
tf.keras.models.save_model(model=model,filepath="C:\\Users\\amith\\Documents\\GitHub\\udemy\\8 - Detecting DeepFakes with ResNet and CNN\\app\\deepfake.h5")
